In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import chardet

In [12]:
# Load CSV files
customers = pd.read_csv('/content/drive/MyDrive/Sprints/Eyouth/ecommerce/customers.csv')
products = pd.read_csv('/content/drive/MyDrive/Sprints/Eyouth/ecommerce/products.csv')
orders = pd.read_csv('/content/drive/MyDrive/Sprints/Eyouth/ecommerce/orders.csv')
order_details = pd.read_csv('/content/drive/MyDrive/Sprints/Eyouth/ecommerce/order_details.csv')
wishlists = pd.read_csv('/content/drive/MyDrive/Sprints/Eyouth/ecommerce/wishlists.csv')

In [13]:
import pandas as pd
import chardet

# Function to detect file encoding
def detect_encoding(file_path):
    with open(file_path, 'rb') as file:
        result = chardet.detect(file.read(10000))  # Read first 10000 bytes
    return result['encoding']

# File paths
file_paths = {
    'customers': '/content/drive/MyDrive/Sprints/Eyouth/ecommerce/customers.csv',
    'products': '/content/drive/MyDrive/Sprints/Eyouth/ecommerce/products.csv',
    'orders': '/content/drive/MyDrive/Sprints/Eyouth/ecommerce/orders.csv',
    'order_details': '/content/drive/MyDrive/Sprints/Eyouth/ecommerce/order_details.csv',
    'wishlists': '/content/drive/MyDrive/Sprints/Eyouth/ecommerce/wishlists.csv'
}

In [14]:
# Load CSVs with detected or fallback encoding
dataframes = {}
for name, path in file_paths.items():
    try:
        # Try UTF-8 first
        df = pd.read_csv(path, encoding='utf-8')
    except UnicodeDecodeError:
        # Detect encoding if UTF-8 fails
        encoding = detect_encoding(path)
        print(f"Detected encoding for {name}.csv: {encoding}")
        try:
            df = pd.read_csv(path, encoding=encoding)
        except Exception as e:
            # Fallback to utf-8-sig or latin1
            print(f"Failed with detected encoding for {name}.csv, trying utf-8-sig: {e}")
            df = pd.read_csv(path, encoding='utf-8-sig')
    dataframes[name] = df
    print(f"Loaded {name}.csv with shape: {df.shape}")

# Assign DataFrames
customers = dataframes['customers']
products = dataframes['products']
orders = dataframes['orders']
order_details = dataframes['order_details']
wishlists = dataframes['wishlists']

Loaded customers.csv with shape: (250, 7)
Loaded products.csv with shape: (600, 8)
Loaded orders.csv with shape: (10200, 5)
Loaded order_details.csv with shape: (30668, 5)
Loaded wishlists.csv with shape: (1007, 4)


In [15]:
orders.info()
order_details.info()
products.info()
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10200 entries, 0 to 10199
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            10200 non-null  int64  
 1   customer_id   10200 non-null  int64  
 2   order_date    10200 non-null  object 
 3   total_amount  10200 non-null  float64
 4   status        10200 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 398.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30668 entries, 0 to 30667
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          30668 non-null  int64  
 1   order_id    30668 non-null  int64  
 2   product_id  30668 non-null  int64  
 3   quantity    30668 non-null  int64  
 4   unit_price  30668 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 1.2 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data colum

In [16]:
# Validate required columns
required_columns = {
    'customers': ['id', 'first_name', 'last_name', 'email', 'phone', 'address', 'registration_date'],
    'products': ['id', 'name', 'price', 'description', 'category_id', 'supplier_id', 'sku', 'stock_quantity'],
    'orders': ['id', 'customer_id', 'order_date', 'total_amount', 'status'],
    'order_details': ['id', 'order_id', 'product_id', 'quantity', 'unit_price'],
    'wishlists': ['customer_id', 'product_id']  # Assuming minimal columns for wishlists
}
for name, df in dataframes.items():
    missing_cols = [col for col in required_columns[name] if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing columns in {name}.csv: {missing_cols}")

In [17]:
# Check for unmatched IDs before merging
unmatched_order_ids = set(order_details['order_id']) - set(orders['id'])
if unmatched_order_ids:
    print(f"Warning: {len(unmatched_order_ids)} order_ids in order_details not found in orders")
unmatched_product_ids = set(order_details['product_id']) - set(products['id'])
if unmatched_product_ids:
    print(f"Warning: {len(unmatched_product_ids)} product_ids in order_details not found in products")

# Data preprocessing: Merge Orders and OrderDetails
purchase_data = order_details.merge(
    orders,
    left_on='order_id',
    right_on='id',
    suffixes=('_detail', '_order')
).merge(
    products,
    left_on='product_id',
    right_on='id',
    suffixes=('', '_product')
)
purchase_data = purchase_data[['customer_id', 'product_id', 'quantity']]

In [18]:
# Check for NaN in purchase_data
print(f"NaN counts in purchase_data:\n{purchase_data.isna().sum()}")
if purchase_data.isna().any().any():
    print("Filling NaN in purchase_data.quantity with 0")
    purchase_data['quantity'] = purchase_data['quantity'].fillna(0)

# Create user-item matrix
user_item_matrix = purchase_data.pivot_table(
    index='customer_id',
    columns='product_id',
    values='quantity',
    fill_value=0,
    aggfunc='sum'  # Sum quantities for duplicate entries
)

NaN counts in purchase_data:
customer_id    0
product_id     0
quantity       0
dtype: int64


In [19]:
# Check for NaN in user_item_matrix
print(f"NaN counts in user_item_matrix:\n{user_item_matrix.isna().sum().sum()}")
if user_item_matrix.isna().any().any():
    print("Filling NaN in user_item_matrix with 0")
    user_item_matrix = user_item_matrix.fillna(0)

# Incorporate wishlist data
wishlist_data = wishlists[['customer_id', 'product_id']].copy()
wishlist_data['quantity'] = 1  # Weight of 1 for wishlist items
print(f"NaN counts in wishlist_data:\n{wishlist_data.isna().sum()}")
if wishlist_data.isna().any().any():
    print("Dropping NaN rows in wishlist_data")
    wishlist_data = wishlist_data.dropna()

NaN counts in user_item_matrix:
0
NaN counts in wishlist_data:
customer_id    0
product_id     0
quantity       0
dtype: int64


In [20]:
wishlist_matrix = wishlist_data.pivot_table(
    index='customer_id',
    columns='product_id',
    values='quantity',
    fill_value=0,
    aggfunc='sum'
)

In [10]:
# Incorporate wishlist data (add 1 to matrix for wishlist items)
wishlist_data = wishlists[['customer_id', 'product_id']].copy()
wishlist_data['quantity'] = 1  # Assume wishlist items have a weight of 1
wishlist_matrix = wishlist_data.pivot_table(
    index='customer_id',
    columns='product_id',
    values='quantity',
    fill_value=0
)

In [21]:
# Combine purchase and wishlist matrices
user_item_matrix = user_item_matrix.add(wishlist_matrix, fill_value=0)

# Final check for NaN in combined matrix
print(f"NaN counts in final user_item_matrix:\n{user_item_matrix.isna().sum().sum()}")
if user_item_matrix.isna().any().any():
    print("Filling NaN in final user_item_matrix with 0")
    user_item_matrix = user_item_matrix.fillna(0)

# Compute item-item similarity using cosine similarity
item_similarity = cosine_similarity(user_item_matrix.T)
item_similarity_df = pd.DataFrame(
    item_similarity,
    index=user_item_matrix.columns,
    columns=user_item_matrix.columns
)

NaN counts in final user_item_matrix:
148
Filling NaN in final user_item_matrix with 0


In [22]:
# Recommendation function
def recommend_products(customer_id, n_recommendations=5):
    if customer_id not in user_item_matrix.index:
        return f"Customer ID {customer_id} not found."

    # Get customer's purchased/wishlisted products
    customer_purchases = user_item_matrix.loc[customer_id]
    purchased_products = customer_purchases[customer_purchases > 0].index

    # Calculate recommendation scores
    scores = item_similarity_df[purchased_products].sum(axis=1)
    scores = scores.sort_values(ascending=False)

    # Exclude already purchased/wishlisted products
    recommended_product_ids = scores.index[~scores.index.isin(purchased_products)][:n_recommendations]

    # Get product details
    recommendations = products[products['id'].isin(recommended_product_ids)][['id', 'name', 'price']]
    recommendations = recommendations.rename(columns={'id': 'product_id'})

    return recommendations

In [24]:
# Example usage
try:
    example_customer_id = customers['id'].iloc[3]  # First customer ID
    print(f"Recommendations for Customer {example_customer_id}:")
    recommendations = recommend_products(example_customer_id)
    print(recommendations)
    # Save recommendations to CSV
    recommendations.to_csv('/content/drive/MyDrive/Sprints/Eyouth/ecommerce/recommendations.csv', index=False)
    print("Recommendations saved to '/content/drive/MyDrive/Sprints/Eyouth/ecommerce/recommendations.csv'")
except IndexError:
    print("No customers found in customers.csv")

Recommendations for Customer 4:
    product_id                                        name   price
16          17           Streamlined multi-state groupware  548.72
39          40                  Up-sized 24hour moratorium  442.83
41          42  Profit-focused dedicated Internet solution  712.00
59          60     Vision-oriented well-modulated database  312.33
62          63              Synchronized systemic firmware  619.69
Recommendations saved to '/content/drive/MyDrive/Sprints/Eyouth/ecommerce/recommendations.csv'
